In [1]:
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.layers import ReLU, Add, Dense, Flatten, Concatenate, ZeroPadding2D, SeparableConv2D
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
#from keras.layers import RandomFlip, RandomRotation, RandomZoom, AutoContrast
from keras_cv.layers import RandomFlip, RandomRotation, RandomZoom, AutoContrast
from keras.optimizers import Adam, RMSprop
from keras.utils import load_img, img_to_array, to_categorical, plot_model
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import Xception

import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [2]:
DIR_TRAIN = "./train/"
DIR_TEST = "./test/"

PATH_TRAIN = "./train.csv"
PATH_TEST = "./test.csv"

In [3]:
data = pd.read_csv(PATH_TRAIN, delimiter=',', header=0)
data.head()

num_classes = int(data['class'].max()) + 1

In [4]:
import shutil

DIR_VAL = "./val/"

def data_for_dataset():
    
    for i in range(num_classes):
        dir = os.path.join(DIR_TRAIN, str(i))
        if not os.path.exists(dir):
            os.mkdir(dir)
        for f in data[data['class']==i].values:
            fn = os.path.join(dir, f[0])
            if not os.path.exists(fn):
                shutil.move(os.path.join(DIR_TRAIN, f[0]), fn)
    
    if not os.path.exists(DIR_VAL):
        os.mkdir(DIR_VAL)
    for i in range(num_classes):
        dir = os.path.join(DIR_VAL, str(i))
        if not os.path.exists(dir):
            os.mkdir(dir)
        ls = sorted(os.listdir(os.path.join(DIR_TRAIN, str(i))))
        for j in range(len(ls)//10):
            ffrom = os.path.join(DIR_TRAIN, str(i), ls[j])
            fto = os.path.join(DIR_VAL, str(i), ls[j])
            if os.path.exists(ffrom) and not os.path.exists(fto):
                shutil.move(ffrom, fto)

#data_for_dataset()


In [5]:
kshape = 128
shape = (kshape, kshape)
xtrain = np.array([img_to_array(load_img(os.path.join(DIR_TRAIN, s), color_mode='rgb', target_size=shape, keep_aspect_ratio=True)) for s in data.ID_img]) / 255.
ytrain = np.array([to_categorical(s, num_classes=num_classes) for s in data['class']])
#ytrain = np.array([int(s) for s in data['class']])

xtrain, xtest, ytrain, ytest = train_test_split(xtrain, ytrain, test_size=.1)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)
#print(xtrain.shape, ytrain.shape)

(4491, 128, 128, 3) (499, 128, 128, 3) (4491, 8) (499, 8)


In [6]:
#from keras.preprocessing.image import ImageDataGenerator

#datagen = ImageDataGenerator(featurewise_center=True, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.1)#horizontal_flip=True
#datagen.fit(xtrain)

In [7]:
def model1():
    l1l2 = L1L2(l1=1e-5, l2=1e-5)
    #, kernel_regularizer=l1l2
    
    inp = Input(shape=(kshape, kshape, 3))

    x = Conv2D(8, 3, padding='valid', kernel_regularizer=l1l2)(inp)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    for k, n in zip([16, 32, 64, 128, 256], [2,2,3,3,3]):
        for i in range(n):
            y = Conv2D(k, 3, padding='same', kernel_regularizer=l1l2)(x)
            y = BatchNormalization()(y)
            y = ReLU()(y)
            y = Conv2D(k, 3, padding='same', kernel_regularizer=l1l2)(y)
            
            x = Conv2D(k, 1, padding='same', kernel_regularizer=l1l2)(x)
            
            if i == n-1:
                y = MaxPooling2D(2)(y)
                x = MaxPooling2D(2)(x)
            
            x = Add()([x,y])
            x = BatchNormalization()(x)
            x = ReLU()(x)
    
    x = Conv2D(512, 3, padding='valid', kernel_regularizer=l1l2)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    #x = Flatten()(x)
    x = GlobalMaxPooling2D()(x)

    for i in range(2):
        x = Dense(512, kernel_regularizer=l1l2)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    x = Dense(num_classes, activation='sigmoid', kernel_regularizer=l1l2)(x)

    model = Model(inp, x)

    return model, './ulit1.h5'

In [8]:
def model2():
    l1l2 = L1L2(l1=1e-5, l2=1e-5)
    #, kernel_regularizer=l1l2
    
    inp = Input(shape=(kshape, kshape, 3))
    x = BatchNormalization()(inp)

    #x = RandomFlip('vertical')(x)
    #x = RandomRotation(0.3)(x)
    #x = RandomZoom(.2, .2)(x)
    #x = AutoContrast([0, 1])(x)
    
    for k in [64, 128, 256]:
        x = Conv2D(k, 3, strides=2, padding='valid', kernel_regularizer=l1l2)(x)
        #x = SeparableConv2D (k, 3, strides=2, padding='valid', kernel_regularizer=l1l2, use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    a = []
    for i in range(16):
        y = Conv2D(64, 3, padding='valid', kernel_regularizer=l1l2)(x)
        #y = SeparableConv2D (64, 3, padding='valid', kernel_regularizer=l1l2, use_bias=False)(x)
        y = BatchNormalization()(y)
        y = ReLU()(y)
        y = Conv2D(64, 3, strides=2, padding='valid', kernel_regularizer=l1l2)(y)
        #y = SeparableConv2D (64, 3, strides=2, padding='valid', kernel_regularizer=l1l2, use_bias=False)(y)
        a.append(y)
    
    x = Concatenate()(a)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(512, 3, padding='valid', kernel_regularizer=l1l2)(x)
    #x = SeparableConv2D (512, 3, padding='valid', kernel_regularizer=l1l2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    #x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)

    for i in range(2):
        x = Dense(512, kernel_regularizer=l1l2)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inp, x)
    model.compile(optimizer=RMSprop(learning_rate=1e-4), loss='categorical_crossentropy', metrics='accuracy')

    return model, './ulite2.h5'

In [9]:
def model3():
    base_model = Xception(include_top=False, input_shape=(kshape, kshape, 3), pooling='avg')
    base_model.trainable = True

    inputs = Input(shape=(kshape,kshape, 3))
    x = base_model(inputs, training=True)
    
    for i in range(2):
        x = Dense(256)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
    
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=x)
    
    return model, './ulite3.h5'

In [19]:
es = EarlyStopping(patience=10, verbose=0, restore_best_weights=True)
rop = ReduceLROnPlateau(patience=8, min_lr=1e-10, verbose=0)

def make_model(model, path_weights, n):
    model.compile(optimizer=Adam(learning_rate=5e-3), loss='categorical_crossentropy', metrics='accuracy')
    mcp = ModelCheckpoint(path_weights, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True)
    for i in range(n):
        try:
            model.load_weights(path_weights)
        except:
            print('mlyaa')
        history = model.fit(xtrain, ytrain, batch_size=100, epochs=500, verbose=0, validation_data=(xtest, ytest), callbacks=[es, mcp, rop])
        #print(history.history['loss'][-1], history.history['accuracy'][-1], history.history['val_loss'][-1], history.history['val_accuracy'][-1])
    model.load_weights(path_weights)
    model.evaluate(xtest, ytest)
    return model

m1, w1 = model1()
m1 = make_model(m1, w1, 2)

m2, w2 = model2()
m2 = make_model(m2, w2, 2)

m3, w3 = model3()
m3 = make_model(m3, w3, 2)


16/16 [==============================] - 1s 53ms/step - loss: 0.0055 - accuracy: 0.9980


In [20]:
data = pd.read_csv(PATH_TEST, delimiter=',', header=0)
xval = np.array([img_to_array(load_img(os.path.join(DIR_TEST, s), color_mode='rgb', target_size=shape, keep_aspect_ratio=True)) for s in data.ID_img]) / 255.


In [24]:
preds = []
for i in range(3):
    preds.append(m1.predict(xval))
for i in range(3):
    preds.append(m2.predict(xval))
for i in range(3):
    preds.append(m3.predict(xval))

pred = []
for p in preds:
    if len(pred) == 0:
        pred = p
    else:
        pred += p

pred = [np.argmax(i) for i in pred]

res = []
for i in range(len(pred)):
    p = pred[i]
    a = []
    eqval = True
    for j in preds:
        a.append(np.argmax(j[i]))
        eqval = eqval and (np.argmax(j[i])==p)
    if not eqval:
        print(i, p, a)
        res.append(a)


67/67 [==============================] - 3s 49ms/step
0 5 [5, 4, 4, 5, 5, 5, 5, 5, 5]
1 5 [5, 2, 2, 4, 4, 4, 5, 5, 5]
12 2 [2, 7, 7, 2, 2, 2, 2, 2, 2]
13 5 [5, 7, 7, 5, 5, 5, 5, 5, 5]
18 4 [4, 4, 4, 4, 4, 4, 3, 3, 3]
20 4 [4, 4, 4, 4, 4, 4, 1, 1, 1]
24 6 [6, 6, 6, 6, 6, 6, 7, 7, 7]
25 1 [1, 2, 2, 0, 0, 0, 1, 1, 1]
26 7 [7, 2, 2, 7, 7, 7, 0, 0, 0]
31 4 [4, 4, 4, 0, 0, 0, 4, 4, 4]
33 4 [4, 4, 4, 4, 4, 4, 5, 5, 5]
36 1 [1, 1, 1, 1, 1, 1, 4, 4, 4]
38 3 [3, 2, 2, 3, 3, 3, 3, 3, 3]
39 5 [5, 3, 3, 5, 5, 5, 5, 5, 5]
40 2 [2, 2, 2, 2, 2, 2, 7, 7, 7]
45 7 [7, 2, 2, 3, 3, 3, 7, 7, 7]
86 4 [4, 3, 3, 5, 5, 5, 4, 4, 4]
87 4 [4, 0, 0, 4, 4, 4, 4, 4, 4]
88 1 [1, 3, 3, 1, 1, 1, 1, 1, 1]
90 4 [4, 3, 3, 4, 4, 4, 4, 4, 4]
98 1 [1, 0, 0, 1, 1, 1, 1, 1, 1]
113 2 [2, 1, 1, 2, 2, 2, 2, 2, 2]
124 1 [1, 1, 1, 1, 1, 1, 0, 0, 0]
140 4 [4, 4, 4, 4, 4, 4, 1, 1, 1]
142 5 [5, 5, 5, 2, 2, 2, 5, 5, 5]
145 4 [4, 4, 4, 4, 4, 4, 1, 1, 1]
150 4 [4, 4, 4, 4, 4, 4, 2, 2, 2]
151 2 [2, 2, 2, 5, 5, 5, 2, 2, 2]
161 2 [2, 1, 1, 2

In [25]:
del(data['class'])
data['class'] = pred
data.head(20)

,ID_img,class
0,0.jpg,5
1,1.jpg,5
2,2.jpg,1
3,3.jpg,1
4,4.jpg,6
5,5.jpg,3
6,6.jpg,7
7,7.jpg,3
8,8.jpg,3
9,9.jpg,5


In [14]:
data.to_csv('./submit.csv', index=False)